In [ ]:
import gensim 
from gensim.models import Word2Vec , KeyedVectors

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

vec_king = wv['king']       

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
vec_king.shape

(300,)

In [277]:
import pandas as pd 
messages = pd.read_csv(r'C:\Learn_AI\Model\Email_Spam\SMSSpamCollection.txt' , sep = '\t' , names = ['label' , 'message']) 

In [333]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [334]:
messages.shape

(5572, 2)

In [335]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [348]:
from nltk.corpus import stopwords
import re
corpus = []
stopwords = stopwords.words('english')
for i in range(0 , len(messages)):
    review = re.sub('[^a-zA-Z]' , ' ' , messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords]
    review = ' '.join(review)
    if review.strip():
        corpus.append(review)
    


In [349]:
len(corpus)

5564

In [350]:
from nltk import sent_tokenize 
from gensim.utils import simple_preprocess

In [356]:
words = []
for sent in corpus : 
    sent_token = sent_tokenize(sent)
    for sent in sent_token : 
        processed_sent = simple_preprocess(sent)
        if processed_sent:
            words.append(processed_sent)
        

In [ ]:
len(words)

5559

In [358]:
import gensim

In [359]:
model= gensim.models.Word2Vec(words , min_count= 1)
model.wv.index_to_key

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'day',
 'ok',
 'free',
 'know',
 'come',
 'like',
 'good',
 'time',
 'got',
 'love',
 'text',
 'want',
 'send',
 'need',
 'one',
 'txt',
 'today',
 'going',
 'stop',
 'home',
 'lor',
 'sorry',
 'see',
 'still',
 'mobile',
 'take',
 'back',
 'da',
 'reply',
 'dont',
 'think',
 'tell',
 'week',
 'hi',
 'phone',
 'new',
 'later',
 'please',
 'pls',
 'co',
 'msg',
 'min',
 'night',
 'make',
 'dear',
 'message',
 'well',
 'say',
 'thing',
 'much',
 'great',
 'claim',
 'hope',
 'oh',
 'hey',
 'give',
 'number',
 'happy',
 'friend',
 'work',
 'wat',
 'yes',
 'way',
 'www',
 'let',
 'prize',
 'right',
 'tomorrow',
 'already',
 'tone',
 'ask',
 'win',
 'said',
 'amp',
 'cash',
 'life',
 'yeah',
 'im',
 'really',
 'babe',
 'meet',
 'find',
 'miss',
 'morning',
 'service',
 'uk',
 'last',
 'year',
 'thanks',
 'care',
 'com',
 'would',
 'anything',
 'lol',
 'nokia',
 'also',
 'every',
 'feel',
 'keep',
 'sure',
 'pick',
 'urgent',
 'sent',
 'contact',


In [360]:
model.corpus_count

5559

In [361]:
model.wv.similar_by_word('good')

[('like', 0.9996698498725891),
 ('got', 0.9996692538261414),
 ('come', 0.9996656179428101),
 ('back', 0.9996640086174011),
 ('tell', 0.9996623992919922),
 ('much', 0.999657154083252),
 ('keep', 0.9996411204338074),
 ('today', 0.9996395707130432),
 ('thing', 0.9996395111083984),
 ('day', 0.9996383190155029)]

In [362]:
def avg_word2vec(doc):
    if not doc:
        return None  # hoặc np.zeros(model.vector_size) nếu bạn muốn vector mặc định
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if not vectors:
        return None
    else :
        return np.mean(vectors, axis=0)

In [364]:
from tqdm import tqdm 
import numpy as np 

In [365]:
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  0%|          | 0/5559 [00:00<?, ?it/s]

100%|██████████| 5559/5559 [00:00<00:00, 7501.49it/s]


In [366]:
len(X)

5559

In [367]:
len(X)

5559

In [ ]:
# # In ra các phần tử không phải là numpy ndarray
# for i, vec in enumerate(X):
#     if not isinstance(vec, np.ndarray):
#         print(f"Phần tử X[{i}] không phải là numpy array: {vec}")


In [370]:
X_new = np.array(X)

In [371]:
X_new.shape

(5559, 100)

In [385]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [386]:
len(words[0])

14

In [ ]:
y = messages[list(map(lambda x: len(x)>0 ,corpus))]
y=pd.get_dummies(y['label'])
y=y.iloc[:,0].values

ValueError: Item wrong length 5564 instead of 5572.

In [393]:
# Lọc các phần tử trống trong corpus, chỉ giữ lại những phần tử có nội dung
valid_indices = [i for i, text in enumerate(words) if len(text.strip()) > 0]

# Lọc lại cả messages và label theo chỉ số hợp lệ
messages_filtered = messages.iloc[valid_indices]
corpus_filtered = [corpus[i] for i in valid_indices]

# Kiểm tra lại độ dài
print(len(messages_filtered), len(corpus_filtered))

# Tiến hành với dữ liệu đã lọc
y = messages_filtered['label']
y = pd.get_dummies(y)
y = y.iloc[:, 0].values
len(y)

AttributeError: 'list' object has no attribute 'strip'

In [376]:
len(y)

5564

In [80]:
y.shape

(5564,)

In [81]:
messages.shape

(5572, 2)

In [74]:
y.shape

(5572,)

In [ ]:
# [[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus, messages['message']) if i<1]

[]

In [84]:
len(X)

5564

In [98]:
df = pd.DataFrame()
for i in range(0 , len(X)):
    df = pd.DataFrame(np.array(X))

In [394]:
from nltk.corpus import stopwords
import re
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess
import numpy as np
from tqdm import tqdm
import pandas as pd

# Tạo corpus và lưu chỉ số hợp lệ
corpus = []
valid_indices = []
stopwords = stopwords.words('english')

for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords]
    review = ' '.join(review)
    if review.strip():
        corpus.append(review)
        valid_indices.append(i)

print(f"Số tài liệu trong corpus: {len(corpus)}")
print(f"Số chỉ số hợp lệ: {len(valid_indices)}")

# Tạo words
words = []
for sent in corpus:
    processed_sent = simple_preprocess(sent)
    if processed_sent:
        words.append(processed_sent)

print(f"Số tài liệu trong words: {len(words)}")

# Hàm avg_word2vec
def avg_word2vec(doc):
    if not doc:
        return np.zeros(model.vector_size)
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Tạo X
X = []
for i in tqdm(range(len(words))):
    vector = avg_word2vec(words[i])
    X.append(vector)

# Chuyển X thành mảng NumPy
X_new = np.array(X)
print("X_new shape:", X_new.shape)

# Tạo y
filtered_messages = messages.iloc[valid_indices]
y = pd.get_dummies(filtered_messages['label'])
y = y.iloc[:, 0].values
print("y shape:", y.shape)

Số tài liệu trong corpus: 5564
Số chỉ số hợp lệ: 5564
Số tài liệu trong words: 5559


100%|██████████| 5559/5559 [00:01<00:00, 5385.56it/s]


X_new shape: (5559, 100)
y shape: (5564,)


In [395]:
from nltk.corpus import stopwords
import re
from gensim.utils import simple_preprocess
import numpy as np
from tqdm import tqdm
import pandas as pd

# Hàm avg_word2vec
def avg_word2vec(doc):
    if not doc:
        return np.zeros(model.vector_size)
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Tạo corpus và lưu chỉ số hợp lệ
corpus = []
valid_indices = []
stopwords = stopwords.words('english')

for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords]
    review = ' '.join(review)
    if review.strip():
        corpus.append(review)
        valid_indices.append(i)

print(f"Số tài liệu trong corpus: {len(corpus)}")
print(f"Số chỉ số hợp lệ: {len(valid_indices)}")

# Tạo words và theo dõi chỉ số
words = []
word_indices = []
for i, sent in enumerate(corpus):
    processed_sent = simple_preprocess(sent)
    if processed_sent:
        words.append(processed_sent)
        word_indices.append(i)

print(f"Số tài liệu trong words: {len(words)}")
print(f"Số chỉ số trong word_indices: {len(word_indices)}")

# Lọc valid_indices dựa trên word_indices
final_indices = [valid_indices[i] for i in word_indices]
print(f"Số chỉ số trong final_indices: {len(final_indices)}")

# Tạo X
X = []
for i in tqdm(range(len(words))):
    vector = avg_word2vec(words[i])
    X.append(vector)

# Chuyển X thành mảng NumPy
X_new = np.array(X)
print("X_new shape:", X_new.shape)

# Tạo y
filtered_messages = messages.iloc[final_indices]
y = pd.get_dummies(filtered_messages['label'])
y = y.iloc[:, 0].values
print("y shape:", y.shape)

Số tài liệu trong corpus: 5564
Số chỉ số hợp lệ: 5564
Số tài liệu trong words: 5559
Số chỉ số trong word_indices: 5559
Số chỉ số trong final_indices: 5559


100%|██████████| 5559/5559 [00:00<00:00, 6619.39it/s]

X_new shape: (5559, 100)
y shape: (5559,)


In [397]:
import pandas as pd
import numpy as np

# Giả sử X là danh sách các vector numpy
df_list = []

for i in range(len(X)):
    if isinstance(X[i], np.ndarray):
        df_list.append(pd.DataFrame(X[i].reshape(1, -1)))

# Nối tất cả lại thành một DataFrame
df = pd.concat(df_list, ignore_index=True)


In [399]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.101006,0.178412,0.038068,0.036395,-0.014553,-0.299925,0.074482,0.373894,-0.109472,-0.092720,...,0.245752,0.107734,-0.004451,0.031475,0.284606,0.189009,0.075432,-0.188142,0.063267,-0.029535
1,-0.064596,0.122469,0.027225,0.029824,-0.009128,-0.209661,0.053657,0.264830,-0.082356,-0.061662,...,0.175647,0.075699,-0.002119,0.024978,0.198515,0.128957,0.053078,-0.134593,0.041818,-0.021655
2,-0.108396,0.187561,0.037593,0.044506,-0.021592,-0.320681,0.080545,0.390882,-0.113199,-0.097411,...,0.260966,0.113070,-0.003038,0.032464,0.304064,0.204048,0.078931,-0.205568,0.069610,-0.026815
3,-0.144802,0.250865,0.052844,0.056771,-0.017818,-0.437294,0.108185,0.542184,-0.156134,-0.125089,...,0.356820,0.149238,-0.004626,0.044509,0.406431,0.276639,0.106229,-0.271083,0.091896,-0.047556
4,-0.146196,0.250408,0.049327,0.050136,-0.027882,-0.426593,0.109062,0.528430,-0.156746,-0.122816,...,0.352976,0.152568,-0.009524,0.042685,0.409363,0.274939,0.110038,-0.272112,0.091564,-0.045277


In [400]:

df['Output']=y

In [401]:

df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.101006,0.178412,0.038068,0.036395,-0.014553,-0.299925,0.074482,0.373894,-0.109472,-0.092720,...,0.107734,-0.004451,0.031475,0.284606,0.189009,0.075432,-0.188142,0.063267,-0.029535,True
1,-0.064596,0.122469,0.027225,0.029824,-0.009128,-0.209661,0.053657,0.264830,-0.082356,-0.061662,...,0.075699,-0.002119,0.024978,0.198515,0.128957,0.053078,-0.134593,0.041818,-0.021655,True
2,-0.108396,0.187561,0.037593,0.044506,-0.021592,-0.320681,0.080545,0.390882,-0.113199,-0.097411,...,0.113070,-0.003038,0.032464,0.304064,0.204048,0.078931,-0.205568,0.069610,-0.026815,False
3,-0.144802,0.250865,0.052844,0.056771,-0.017818,-0.437294,0.108185,0.542184,-0.156134,-0.125089,...,0.149238,-0.004626,0.044509,0.406431,0.276639,0.106229,-0.271083,0.091896,-0.047556,True
4,-0.146196,0.250408,0.049327,0.050136,-0.027882,-0.426593,0.109062,0.528430,-0.156746,-0.122816,...,0.152568,-0.009524,0.042685,0.409363,0.274939,0.110038,-0.272112,0.091564,-0.045277,True


In [402]:

df.dropna(inplace=True)

In [403]:

df.isnull().sum()

0         0
1         0
2         0
3         0
4         0
         ..
96        0
97        0
98        0
99        0
Output    0
Length: 101, dtype: int64

In [418]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.101006,0.178412,0.038068,0.036395,-0.014553,-0.299925,0.074482,0.373894,-0.109472,-0.092720,...,0.107734,-0.004451,0.031475,0.284606,0.189009,0.075432,-0.188142,0.063267,-0.029535,True
1,-0.064596,0.122469,0.027225,0.029824,-0.009128,-0.209661,0.053657,0.264830,-0.082356,-0.061662,...,0.075699,-0.002119,0.024978,0.198515,0.128957,0.053078,-0.134593,0.041818,-0.021655,True
2,-0.108396,0.187561,0.037593,0.044506,-0.021592,-0.320681,0.080545,0.390882,-0.113199,-0.097411,...,0.113070,-0.003038,0.032464,0.304064,0.204048,0.078931,-0.205568,0.069610,-0.026815,False
3,-0.144802,0.250865,0.052844,0.056771,-0.017818,-0.437294,0.108185,0.542184,-0.156134,-0.125089,...,0.149238,-0.004626,0.044509,0.406431,0.276639,0.106229,-0.271083,0.091896,-0.047556,True
4,-0.146196,0.250408,0.049327,0.050136,-0.027882,-0.426593,0.109062,0.528430,-0.156746,-0.122816,...,0.152568,-0.009524,0.042685,0.409363,0.274939,0.110038,-0.272112,0.091564,-0.045277,True


In [419]:
df['Output']

0        True
1        True
2       False
3        True
4        True
        ...  
5554    False
5555     True
5556     True
5557     True
5558     True
Name: Output, Length: 5559, dtype: bool

In [421]:
X=df.drop('Output' , axis =1)

In [422]:

X.isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
95    0
96    0
97    0
98    0
99    0
Length: 100, dtype: int64

In [423]:

y=df['Output']

In [424]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [425]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1056,-0.115427,0.193311,0.042165,0.043325,-0.016972,-0.336736,0.084784,0.407284,-0.121291,-0.099030,...,0.267831,0.116776,-0.002814,0.031728,0.318727,0.208149,0.087529,-0.210563,0.065856,-0.030949
3653,-0.240423,0.399516,0.067515,0.067017,-0.017394,-0.655654,0.170937,0.823267,-0.242948,-0.186911,...,0.522517,0.220652,-0.008733,0.057624,0.645511,0.406607,0.182923,-0.419180,0.130495,-0.060240
1594,-0.152597,0.264851,0.059544,0.058994,-0.029935,-0.451327,0.116537,0.557030,-0.155976,-0.151209,...,0.369205,0.162782,-0.007344,0.049317,0.450881,0.295895,0.117048,-0.294788,0.103804,-0.034624
3600,-0.116560,0.211005,0.038217,0.044964,-0.017929,-0.360837,0.089740,0.448382,-0.125457,-0.108335,...,0.301507,0.121870,-0.001795,0.036339,0.341452,0.227116,0.096014,-0.222289,0.071595,-0.031540
737,-0.146583,0.244773,0.051527,0.056351,-0.022399,-0.417671,0.101757,0.508737,-0.149513,-0.127924,...,0.343639,0.144097,-0.001604,0.039154,0.394968,0.263099,0.100593,-0.267190,0.087380,-0.037854


In [426]:

y_train

1056     True
3653     True
1594    False
3600     True
737     False
        ...  
5550     True
3360     True
1928     True
2382     True
1004    False
Name: Output, Length: 4447, dtype: bool

In [449]:

from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()


In [450]:

classifier.fit(X_train,y_train)

RandomForestClassifier()

In [451]:

y_pred=classifier.predict(X_test)

In [452]:

from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

0.9469424460431655


In [453]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.90      0.70      0.79       157
        True       0.95      0.99      0.97       955

    accuracy                           0.95      1112
   macro avg       0.93      0.84      0.88      1112
weighted avg       0.95      0.95      0.94      1112

